In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import s3fs
xr.set_options(display_style='html')
import intake
import cftime
import matplotlib.pyplot as plt
import numpy as np
from netCDF4 import Dataset
from matplotlib.colors import LogNorm
import matplotlib.path as mpath
from read_pangeo import read_pangeo, read_to_detect
from importcmip import aod245_8500

In [ ]:
#load and merge data
arh = xr.open_dataset('historical20002014_AR_detection.nc')
ar245 = xr.open_dataset('ssp24520852100_AR_detection.nc')
ar370 = xr.open_dataset('ssp37020852099_AR_detection.nc')
ar585 = xr.open_dataset('ssp58520852099_AR_detection.nc')

cmh = read_pangeo(2000, 2014, 'historical')
dh = cmh.merge(aodh.drop(('lat_bnds', 'time_bnds', 'lon_bnds')))

cm245 = read_pangeo(2085, 2099, 'ssp245')
d245= cm245.merge(aod245.drop(('lat_bnds', 'time_bnds', 'lon_bnds')))

cm370 = read_pangeo(2085, 2099, 'ssp370')
cm370b = read_370()
d370 = cm370.merge(aod370.drop(('lat_bnds', 'time_bnds', 'lon_bnds')))
d370 = d370.merge(cm370b)

cm585 = read_pangeo(2085, 2099, 'ssp585')
d585 = cm585.merge(aod585.drop(('lat_bnds', 'time_bnds', 'lon_bnds')))



In [ ]:
# slice to poles and only until pressure levels where AR can be detected
n245 = d245.sel(lat = slice(50,90),plev=slice(100000, 25000))
s245 =d245.sel(lat = slice(-90,-50),plev=slice(100000, 25000))

n370 = d370.sel(lat = slice(50,90),plev=slice(100000, 25000))
s370 =d370.sel(lat = slice(-90,-50),plev=slice(100000, 25000))

n585 = d585.sel(lat = slice(50,90),plev=slice(100000, 25000))
s585 =d585.sel(lat = slice(-90,-50),plev=slice(100000, 25000))

nh = dh.sel(lat = slice(50,90), plev=slice(100000, 25000))
sh =dh.sel(lat = slice(-90,-50), plev=slice(100000, 25000))


# mask data for being inside or outside of a atmospheric river
# arctic
masked_n245 = n245.where(ar245['ivt']==True)
masked_n370 = n370.where(ar370['ivt']==True)
masked_n585 = n585.where(ar585['ivt']==True)
masked_nh = nh.where(arh['ivt']==True)

neg_n245 = n245.where(ar245['ivt']==False)
neg_n370 = n370.where(ar370['ivt']==False)
neg_n585 = n585.where(ar585['ivt']==False)
neg_nh = nh.where(arh['ivt']==False)

# antarctic
masked_s245 = s245.where(ar245['ivt']==True)
masked_s370 = s370.where(ar370['ivt']==True)
masked_s585 = s585.where(ar585['ivt']==True)
masked_sh = sh.where(arh['ivt']==True)

neg_s245 = s245.where(ar245['ivt']==False)
neg_s370 = s370.where(ar370['ivt']==False)
neg_s585 = s585.where(ar585['ivt']==False)
neg_sh = sh.where(arh['ivt']==False)

In [ ]:
#treat variables for plotting
#integrate needed humidity
int_nh245 =-1*masked_n245['hus'].integrate('plev')
int_sh245 =-1*masked_s245['hus'].integrate('plev')
int_nh370 =-1*masked_n370['hus'].integrate('plev')
int_sh370 =-1*masked_s370['hus'].integrate('plev')
int_nh585 =-1*masked_n585['hus'].integrate('plev')
int_sh585 =-1*masked_s585['hus'].integrate('plev')
int_nhh =-1*masked_nh['hus'].integrate('plev')
int_shh =-1*masked_sh['hus'].integrate('plev')

int_nh245 = int_nh245.values.flatten() 
int_sh245 = int_sh245.values.flatten()
int_nh370 = int_nh370.values.flatten()
int_sh370 = int_sh370.values.flatten()
int_nh585 = int_nh585.values.flatten()
int_sh585 = int_sh585.values.flatten()
int_nhh = int_nhh.values.flatten()
int_shh = int_shh.values.flatten()

int_nh245 = int_nh245[~np.isnan(mh)]
int_sh245 = int_sh245[~np.isnan(mh)]
int_nh370 = int_nh370[~np.isnan(mh)]
int_sh370 = int_sh370[~np.isnan(mh)]
int_nh585 = int_nh585[~np.isnan(mh)]
int_sh585 = int_sh585[~np.isnan(mh)]
int_nhh = int_nhh[~np.isnan(mh)]
int_shh = int_shh[~np.isnan(mh)]


In [ ]:

#axs = axs.flatten()
#fig.suptitle('', fontsize=16)
def freq_hist(var_name, var245, var370, var585, varhist):
    fig, axs = plt.subplots(figsize=(10,6))
    fig.suptitle(str(var_name), fontsize=16)
    axs.hist(var245, weights = np.zeros_like(var245)+1./var245.size, 
           alpha = 0.5, color = "g", label = "ssp245", bins=30)
    axs.hist(var370, weights = np.zeros_like(var370)+1./var370.size, 
           alpha = 0.3, color = "b", label = "ssp370", bins=30)
    axs.hist(var585, weights = np.zeros_like(var585)+1./var585.size, 
           alpha = 0.5, color = "y", label = "ssp585", bins=30)
    axs.hist(varhist, weights = np.zeros_like(varhist)+1./int_nhh.size, 
           alpha = 0.2, color = "k", label = "historical", bins=30)
    axs.label()
    


In [ ]:
'''# for frequency plots slice for plev >25000 and integrate over plev
mpl =marc.sel(plev=slice(100000, 25000))
pl =amarc.sel(plev=slice(100000, 25000)).where(arc['va'] > 0)

# wind
mplva =-1*mpl['va'].integrate('plev')
plva =-1*pl['va'].integrate('plev')

# humidity
mplh =-1*mpl['hus'].integrate('plev')
plh =-1*pl['hus'].integrate('plev')

mv = mplva.values.flatten()
mv = mv[~np.isnan(mv)]
v = plva.values.flatten()
v= v[~np.isnan(v)]
mh = mplh.values.flatten()
mh = mh[~np.isnan(mh)]
h = plh.values.flatten()
h = h[~np.isnan(h)]
mcc = pl['clt'].values.flatten()
mcc = mcc[~np.isnan(mcc)]
cc = mpl['clt'].values.flatten()
cc = cc[~np.isnan(cc)]
mt = mpl['tas'].values.flatten()
mt = mt[~np.isnan(mt)]
t = pl['tas'].values.flatten()
t = t[~np.isnan(t)]
ma = mpl['od550aer'].values.flatten()
ma = ma[~np.isnan(ma)]
a = pl['od550aer'].values.flatten()
a = a[~np.isnan(a)]
mp = mpl['pr'].values.flatten()
mp = mp[~np.isnan(mp)]
mp = mp[mp>0.0000024099]
p = pl['pr'].values.flatten()
p = p[~np.isnan(p)]
p = p[p>0.0000024099] # threshold for averaged precipitation in the sahara in the same unit'''